<a href="https://colab.research.google.com/github/myspip/random_data/blob/master/glove%20and%20keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function

import os
import sys
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from keras.initializers import Constant


BASE_DIR = ''
GLOVE_DIR = os.path.join(BASE_DIR, 'glove.6B')
MAX_SEQUENCE_LENGTH = 1000
MAX_NUM_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2

In [0]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2019-03-25 21:14:04--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2019-03-25 21:14:04--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M  3.31MB/s    in 11m 7s  

2019-03-25 21:25:11 (1.23 MB/s) - ‘glove.6B.zip’ saved [862182613/862182613]



In [0]:
# import os
# import gensim
# from gensim.models.word2vec import Word2Vec # the word2vec model gensim class
# import gzip, shutil

# print(gensim.__version__)

3.6.0


In [0]:
# !wget -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

In [0]:

# with gzip.open('GoogleNews-vectors-negative300.bin.gz', 'r') as f_in, open('GoogleNews-vectors-negative300.bin', 'wb') as f_out:
#   shutil.copyfileobj(f_in, f_out)

In [0]:
# model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)  

In [0]:
# word_vectors = model.wv
# MAX_NB_WORDS = len(word_vectors.vocab)
# MAX_SEQUENCE_LENGTH = 200

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


In [0]:
# print("Number of word vectors: {}".format(len(word_vectors.vocab)))

Number of word vectors: 3000000


In [0]:
import pandas as pd
import numpy as np
import re

In [0]:
train_data = pd.read_csv("https://raw.githubusercontent.com/myspip/random_data/master/Headline/Clean1_TRAIN.csv")
train_data.loc[:, "Headline"].replace(re.compile(r'\b{}\b'.format("company company"), re.IGNORECASE), "company", inplace=True)
train_data.head()

,Id,Sentiment,Headline
0,0,Bad,company thats lot recalls barrons blog
1,1,Neutral,event alert kinaxis customer company company ...
2,2,Bad,company risky autonomous driving plan barr...
3,3,Good,company plans ridehailing service fleet driv...
4,4,Bad,company files k events f


In [0]:
test_data = pd.read_csv("https://raw.githubusercontent.com/myspip/random_data/master/Headline/Clean1_TEST.csv")
test_data.loc[:, "Headline"].replace(re.compile(r'\b{}\b'.format("company company"), re.IGNORECASE), "company", inplace=True)
test_data.head()

,Id,Sentiment,Headline
0,0,Good,building materials producer company raises sa...
1,1,Good,company plans asset sales debt reduction
2,2,Good,company announces increase net income s...
3,3,Good,years opposition mine expansion passes despi...
4,4,Good,digital platform company go lists customers


In [0]:
train_data_list = train_data["Headline"].values
label = train_data["Sentiment"].values

test_data_list = test_data["Headline"].values
test_label = test_data["Sentiment"].values

combined_data_list = np.concatenate((train_data_list,test_data_list), axis = 0)
len(combined_data_list)

2147

In [0]:
transformed_label = [0 if x == "Bad" else (1 if x == "Good" else 2) for x in label]
test_transformed_label = [0 if x == "Bad" else (1 if x == "Good" else 2) for x in test_label]
transformed_label[0:13]


[0, 2, 0, 1, 0, 0, 0, 0, 2, 2, 2, 1, 1]

In [0]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
import numpy as np
MAX_NB_WORDS = 100

tokenizer = Tokenizer(nb_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(combined_data_list)

/usr/local/lib/python3.6/dist-packages/keras_preprocessing/text.py:178: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


In [0]:
sequences = tokenizer.texts_to_sequences(train_data_list)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

labels = to_categorical(np.asarray(transfomed_label))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

x_train = data
y_train = labels

In [0]:
sequences = tokenizer.texts_to_sequences(test_data_list)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

labels = to_categorical(np.asarray(test_transformed_label))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

x_test = data
y_test = labels

Found 4578 unique tokens.
Shape of data tensor: (241, 200)
Shape of label tensor: (241, 3)


In [0]:
embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

In [0]:
re.sub("[^\w]", " ",  train_data.Headline[1]).split()
vocab['event'].count

2999576

In [0]:
from keras.preprocessing.sequence import pad_sequences
vocab = word_vectors.vocab
sequences = [[vocab[t].count for t in re.sub("[^\w]", " ",  headline).split()]
             for headline in train_data["Headline"].values]
test_sequences = [[vocab[t] for t in re.sub("[^\w]", " ",  headline).split()]
                  for headline in test_data["Headline"].values]



KeyError: ignored

In [0]:
word_vectors.vocab[''].count

1319845

In [0]:
# pad
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH, 
                     padding="pre", truncating="post")
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train_df[list_classes].values
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', y.shape)

test_data = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding="pre",
                          truncating="post")
print('Shape of test_data tensor:', test_data.shape)

In [0]:
WV_DIM = 100
nb_words = min(MAX_NB_WORDS, len(word_vectors.vocab))
# we initialize the matrix with random numbers
wv_matrix = (np.random.rand(nb_words, WV_DIM) - 0.5) / 5.0
for word, i in word_index.items():
    if i >= MAX_NB_WORDS:
        continue
    try:
        embedding_vector = word_vectors[word]
        # words not found in embedding index will be all-zeros.
        wv_matrix[i] = embedding_vector
    except:
        pass  

In [0]:
import json

with open("/var/log/colab-jupyter.log", "r") as fo:
  for line in fo:
    print(json.loads(line)['msg'])

Writing notebook server cookie secret to /root/.local/share/jupyter/runtime/notebook_cookie_secret
google.colab serverextension initialized.
Serving notebooks from local directory: /
0 active kernels
The Jupyter Notebook is running at:
http://172.28.0.2:9000/
Use Control-C to stop this server and shut down all kernels (twice to skip confirmation).
Kernel started: f910f69f-f6c3-4929-9bc3-47c4ed2eb019
Adapting to protocol v5.1 for kernel f910f69f-f6c3-4929-9bc3-47c4ed2eb019
Adapting to protocol v5.1 for kernel f910f69f-f6c3-4929-9bc3-47c4ed2eb019
Kernel started: b1c5404a-3825-4f8b-aa5c-56aa8e42eaa5
Adapting to protocol v5.1 for kernel b1c5404a-3825-4f8b-aa5c-56aa8e42eaa5
tcmalloc: large alloc 3644260352 bytes == 0x117398000 @  0x7ff2083f11e7 0x59213c 0x596936 0x5d6527 0x565cda 0x5a3761 0x5a3a5e 0x4c4364 0x502d6f 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x511eca 0x502d6f 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x501b2e 0x59146